In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm

url = "https://www.news.com.au/"

r = requests.get(url)
soup = bs(r.content)
soup

C:\Users\anurag\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\anurag\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy

<!DOCTYPE html>
<html lang="en"><head><title>news.com.au — Australia’s leading news site</title><link href="https://tags.tiqcdn.com/" rel="dns-prefetch"/><link href="https://nca.talk.news.com.au/" rel="dns-prefetch"/><link href="https://tags.news.com.au/" rel="dns-prefetch"/><link href="https://metrics.news.com.au/" rel="dns-prefetch"/><link href="https://cdn.taboola.com/" rel="dns-prefetch"/><meta charset="utf-8"/><meta content="True" name="HandheldFriendly"/><meta content="320" name="MobileOptimized"/><meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/><meta content="origin-when-cross-origin" name="referrer"/><meta content="noarchive" name="robots"/><meta content="#ffffff" name="theme-color"/><meta content="news" name="application-name"/><meta content="GWTbDFKaKnGKyk3q0Na5jIbWHZ4qXn-ibcfMayjSQA0" name="google-site-verification"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/><!-- CTS-1988 smart banner --><meta content="app-id=1454481155, a

In [2]:
div = soup.find_all("div", class_ = "news-tops_group-2")

categories = []
for d in div:
    cat = d.find_all("a", class_ = "storyblock_section g_font-base-s")
    categories.append(cat)
    
categories = [i for n, i in enumerate(categories) if i not in categories[:n]]

categories = categories[0]

len(categories)

categories

[<a class="storyblock_section g_font-base-s" data-tgev="event10" data-tgev-container="tops" data-tgev-label="section-tag" data-tgev-metric="npv" data-tgev-order="5" href="/entertainment/celebrity-life/celebrity-photos">Celebrity Photos</a>,
 <a class="storyblock_section g_font-base-s" data-tgev="event10" data-tgev-container="tops" data-tgev-label="section-tag" data-tgev-metric="npv" data-tgev-order="6" href="/lifestyle/real-life/news-life">News Life</a>,
 <a class="storyblock_section g_font-base-s" data-tgev="event10" data-tgev-container="tops" data-tgev-label="section-tag" data-tgev-metric="npv" data-tgev-order="7" href="/entertainment/celebrity-life/royals">Royals</a>,
 <a class="storyblock_section g_font-base-s" data-tgev="event10" data-tgev-container="tops" data-tgev-label="section-tag" data-tgev-metric="npv" data-tgev-order="8" href="/technology/science/space">Space</a>,
 <a class="storyblock_section g_font-base-s" data-tgev="event10" data-tgev-container="tops" data-tgev-label="se

In [3]:
div = soup.find_all("div", class_ = "news-tops_group-2")

titles = []
for d in div:
    tit = d.find_all("a", class_ = "storyblock_title_link")
    titles.append(tit)
    
titles = [i for n, i in enumerate(titles) if i not in titles[:n]]

titles = titles[0]
len(titles)


10

In [4]:
div = soup.find_all("div", class_ = "news-tops_group-2")

texts=[]
for d in div:
    t = d.find_all("p", class_="storyblock_standfirst g_font-body-s")
    texts.append(t)
    
texts = [i for n, i in enumerate(texts) if i not in texts[:n]]

texts = texts[0]
len(texts)


10

In [5]:
df_dict = {"title": titles, "description": texts, "category": categories}

df = pd.DataFrame(df_dict)

In [6]:
df=df.astype(str)
df

,title,description,category
0,"<a class=""storyblock_title_link"" data-tgev=""ev...","<p class=""storyblock_standfirst g_font-body-s""...","<a class=""storyblock_section g_font-base-s"" da..."
1,"<a class=""storyblock_title_link"" data-tgev=""ev...","<p class=""storyblock_standfirst g_font-body-s""...","<a class=""storyblock_section g_font-base-s"" da..."
2,"<a class=""storyblock_title_link"" data-tgev=""ev...","<p class=""storyblock_standfirst g_font-body-s""...","<a class=""storyblock_section g_font-base-s"" da..."
3,"<a class=""storyblock_title_link"" data-tgev=""ev...","<p class=""storyblock_standfirst g_font-body-s""...","<a class=""storyblock_section g_font-base-s"" da..."
4,"<a class=""storyblock_title_link"" data-tgev=""ev...","<p class=""storyblock_standfirst g_font-body-s""...","<a class=""storyblock_section g_font-base-s"" da..."
5,"<a class=""storyblock_title_link"" data-tgev=""ev...","<p class=""storyblock_standfirst g_font-body-s""...","<a class=""storyblock_section g_font-base-s"" da..."
6,"<a class=""storyblock_title_link"" data-tgev=""ev...","<p class=""storyblock_standfirst g_font-body-s""...","<a class=""storyblock_section g_font-base-s"" da..."
7,"<a class=""storyblock_title_link"" data-tgev=""ev...","<p class=""storyblock_standfirst g_font-body-s""...","<a class=""storyblock_section g_font-base-s"" da..."
8,"<a class=""storyblock_title_link"" data-tgev=""ev...","<p class=""storyblock_standfirst g_font-body-s""...","<a class=""storyblock_section g_font-base-s"" da..."
9,"<a class=""storyblock_title_link"" data-tgev=""ev...","<p class=""storyblock_standfirst g_font-body-s""...","<a class=""storyblock_section g_font-base-s"" da..."


In [7]:
title_cleaned = []
for u in df["title"]:
    users = u.split(">")[1]
    user = users.split("<")[0]
    title_cleaned.append(user)
title_cleaned

['Blac Chyna’s jaw-dropping transformation',
 '‘Get a real job’: Woman erupts at influencers',
 '‘She’s gone’: Weird Meghan trend revealed',
 'Confusion as Neptune suddenly changes',
 'Actor parties with supermodels in Mykonos',
 'Reason expat refuses to move back to US',
 'Aussie plumber’s vile find in hotel drain',
 'Extent of star’s shocking injury revealed',
 '‘Yum’: Macca’s confirms iconic McFlurry',
 '‘One true love’: best selling ‘sexy’ bra wows']

In [8]:
description_cleaned = []
for u in df["description"]:
    users = u.split(">")[1]
    user = users.split("<")[0]
    description_cleaned.append(user)
description_cleaned

['Rob Kardashian’s ex-fiance Blac Chyna has shown off the staggering results of her new fitness regimen, after reversing her cosmetic procedures.',
 'This is the shocking moment a furious woman unleashes on a group of influencers after witnessing a divisive act on a Gold Coast street.',
 'As the Duke and Duchess of Sussex settle into their US lives, there is one fundamental difference between the couple that seems unsolvable.',
 'Scientists were stunned to find out Neptune’s cloud cover had nearly vanished – and immediately set out to solve the mystery.',
 'Looks like he’s enjoying himself. Actor Glen Powell has been spotted partying with a bevy of supermodels on a yacht in Greece. ',
 'An American expat has revealed why you couldn’t pay her a million dollars to move back to the US.',
 'A plumber has been labelled a ‘hero’ after making a foul discovery while unblocking a drain in an Australian hotel room bathroom.',
 'The Manly Sea Eagles have confirmed the extent of Reuben Garrick’s i

In [9]:
category_cleaned = []
for u in df["category"]:
    users = u.split(">")[1]
    user = users.split("<")[0]
    category_cleaned.append(user)
category_cleaned

['Celebrity Photos',
 'News Life',
 'Royals',
 'Space',
 'Celebrity Photos',
 'Travel Stories',
 'Accommodation',
 'NRL',
 'Eat',
 'BEST OF ']

In [10]:
df_dict = {"title": title_cleaned, "description": description_cleaned, "category": category_cleaned}

df = pd.DataFrame(df_dict)

df

,title,description,category
0,Blac Chyna’s jaw-dropping transformation,Rob Kardashian’s ex-fiance Blac Chyna has show...,Celebrity Photos
1,‘Get a real job’: Woman erupts at influencers,This is the shocking moment a furious woman un...,News Life
2,‘She’s gone’: Weird Meghan trend revealed,As the Duke and Duchess of Sussex settle into ...,Royals
3,Confusion as Neptune suddenly changes,Scientists were stunned to find out Neptune’s ...,Space
4,Actor parties with supermodels in Mykonos,Looks like he’s enjoying himself. Actor Glen P...,Celebrity Photos
5,Reason expat refuses to move back to US,An American expat has revealed why you couldn’...,Travel Stories
6,Aussie plumber’s vile find in hotel drain,A plumber has been labelled a ‘hero’ after mak...,Accommodation
7,Extent of star’s shocking injury revealed,The Manly Sea Eagles have confirmed the extent...,NRL
8,‘Yum’: Macca’s confirms iconic McFlurry,It might be one of the most anticipated releas...,Eat
9,‘One true love’: best selling ‘sexy’ bra wows,This bra has been a bestsellers for years and ...,BEST OF


In [11]:
df=df.replace('realestate.com.au','Real Estate')
df=df.replace('NSW / ACT Courts &amp; Law','Law')
df=df.replace('VIC Courts &amp; Law','Law')
df

,title,description,category
0,Blac Chyna’s jaw-dropping transformation,Rob Kardashian’s ex-fiance Blac Chyna has show...,Celebrity Photos
1,‘Get a real job’: Woman erupts at influencers,This is the shocking moment a furious woman un...,News Life
2,‘She’s gone’: Weird Meghan trend revealed,As the Duke and Duchess of Sussex settle into ...,Royals
3,Confusion as Neptune suddenly changes,Scientists were stunned to find out Neptune’s ...,Space
4,Actor parties with supermodels in Mykonos,Looks like he’s enjoying himself. Actor Glen P...,Celebrity Photos
5,Reason expat refuses to move back to US,An American expat has revealed why you couldn’...,Travel Stories
6,Aussie plumber’s vile find in hotel drain,A plumber has been labelled a ‘hero’ after mak...,Accommodation
7,Extent of star’s shocking injury revealed,The Manly Sea Eagles have confirmed the extent...,NRL
8,‘Yum’: Macca’s confirms iconic McFlurry,It might be one of the most anticipated releas...,Eat
9,‘One true love’: best selling ‘sexy’ bra wows,This bra has been a bestsellers for years and ...,BEST OF


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm

le = preprocessing.LabelEncoder()
df["category_num"] = le.fit_transform(df["category"])
df

,title,description,category,category_num
0,Blac Chyna’s jaw-dropping transformation,Rob Kardashian’s ex-fiance Blac Chyna has show...,Celebrity Photos,2
1,‘Get a real job’: Woman erupts at influencers,This is the shocking moment a furious woman un...,News Life,5
2,‘She’s gone’: Weird Meghan trend revealed,As the Duke and Duchess of Sussex settle into ...,Royals,6
3,Confusion as Neptune suddenly changes,Scientists were stunned to find out Neptune’s ...,Space,7
4,Actor parties with supermodels in Mykonos,Looks like he’s enjoying himself. Actor Glen P...,Celebrity Photos,2
5,Reason expat refuses to move back to US,An American expat has revealed why you couldn’...,Travel Stories,8
6,Aussie plumber’s vile find in hotel drain,A plumber has been labelled a ‘hero’ after mak...,Accommodation,0
7,Extent of star’s shocking injury revealed,The Manly Sea Eagles have confirmed the extent...,NRL,4
8,‘Yum’: Macca’s confirms iconic McFlurry,It might be one of the most anticipated releas...,Eat,3
9,‘One true love’: best selling ‘sexy’ bra wows,This bra has been a bestsellers for years and ...,BEST OF,1


In [13]:
df["description"][0]

'Rob Kardashian’s ex-fiance Blac Chyna has shown off the staggering results of her new fitness regimen, after reversing her cosmetic procedures.'

In [16]:
X_train = df["description"]
y_train = df["category"]

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(X_train)

clf = svm.SVC(kernel="linear")
clf.fit(train_x_vectors, y_train)

test_x_vectorized = vectorizer.transform(["The Duchess of Sussex"])
prediction = clf.predict(test_x_vectorized)
print(prediction)

['Celebrity Photos']
